In [5]:
###### Maintenance Script for Updating and Maintaining Chemistry Structure Annotations ######

### Step 1: Initiation ###

from rdkit import Chem, rdBase
from rdkit.Chem import Draw,rdDepictor
from rdkit.Chem import AllChem, Descriptors, DataStructs
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.Chem.Scaffolds import rdScaffoldNetwork
from datetime import datetime
# import math
# import matplotlib as mpl
# from matplotlib import pyplot as plt
# import os
# import pyvis
# from pyvis.network import Network
# import inspect
import psycopg2
from collections import defaultdict
import collections
import sys
sys.path.append('/mnt/raid1/Python_projects')
sys.path.append('/mnt/raid1/Python_projects/pkg_mod')

##### Custom Modules #####
import chemistry_psql as cp
import fingerprinter as fpt
import db_connector as db

##### Parameters ######

#db_host = db.db_host
#host_main = "localhost"
limit = 2500 # Row limit for SQL 
limit = str(limit)

# DATABASE Connections

#print(db.db_host)
#print(db.chem_db)
#print(db.chembl_db)

# Chemistry Database
#db_chem = psycopg2.connect(host = "localhost", dbname=db.chem_db, user="postgres", password="postgres")
db_chem = psycopg2.connect(host = db.db_host, dbname= db.chem_db, user="postgres", password="postgres")
# create a cursor
cur = db_chem.cursor()
# execute a statement
cur.execute('SELECT version()')
db_ver = cur.fetchone()
print('PostgreSQL database version:', db_ver) 
#display the PostgreSQL database server version


print('RDKit version: ',rdBase.rdkitVersion)
# print('MatplotLib version:', mpl.__version__)
# print('Pyvis version:', pyvis.__version__)
print('Start Time: ',datetime.now())

###################################################################################################################

### Step 2: Generate Fingerprints for New Structures ("no fingerprints") ###


# SELECT Structures without fingerprints

# db_chem = psycopg2.connect(host = "localhost", dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_fingerprint fp WHERE f.structure_id = fp.structure_id)) \
LIMIT '+ limit 
cur.execute(sql) 

mollist = cur.fetchall()
print()
print('##### Structures without Fingerprints #####')
print()
print(len(mollist),'new Structures')
print(mollist)


# generate Morgan fingerprints for molecules

fp_key = {}
fp_val = []
fp_dict = collections.defaultdict(list)
fp_allmol = []
fp_smi_list = {}

for mol in mollist:
    print()
    print('Molecule : ',mol)
    # print(mol[0], mol[1])
    m = Chem.MolFromSmiles(mol[1])

    # Generate Fingerprint SMILES and SMILES list
    fp_smi = fpt.FingerprintToSmiles(m, 3)
    print('Fingerprint : ',fp_smi)    
    for f in fp_smi: 
        fp_smi_list[f[0]] = f[1]
             
    # Generate Fingerprint Footprints:
    big ={}
    fp = AllChem.GetMorganFingerprint(m,3, bitInfo=big)
    # print(len(big))
    # print(big)
    # print(fp)
    for fpb,v in big.items():
        ma = []
        fp_dict[fpb].append(len(v))
        # print(mol[0],fpb, len(v),v)
        ma.append(mol[0])
        ma.append(fpb)
        ma.append(len(v))
        # print(ma)
        fp_allmol.append(ma)
        
print()
print("All Fingerprints by molecule")
print(fp_allmol)
        
print()        
print(fp_smi_list)
print(len(fp_smi_list),'SMILES')

for fpb,v in fp_smi_list.items():
    
    print(fpb,' - ',v)
    
        # key = fbp
        #fp_dict[fpb].append(len(v))
                
print()        
# print(fp_dict)
print(len(fp_dict),'fingerprints')

for fpb,v in fp_dict.items():
    print(fpb,' - ',len(v),' - ',v)
        # key = fbp
        #fp_dict[fpb].append(len(v))
        
###################################################################################################################

### Step 3: Insert new fingerprints into fingerprint table ###

print()
print('##### Insert New Fingerprints #####')
print()

# db_chem = psycopg2.connect(host = "localhost", dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public.fingerprints WHERE fp_type = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(len(fpdblist), "existing fingerprints")
# print(fpdblist)
# print(fp_smi_list)

print()
if len(fp_smi_list) == 0:
    print('No new fingerprint inserts')
else:
    print("New inserted fingerprints")
print()

sql = 'INSERT INTO public.fingerprints (fp_smiles, fp_number, fp_type) VALUES(%s, %s, 1);'

for fpb,v in fp_smi_list.items():
    smi_fl = 0
    for i in range(0,len(fpdblist)):
        if fpb == fpdblist[i][3]:
            smi_fl = 1

    if smi_fl == 0:
        print(fpb,' - ',v, smi_fl, "inserted")
        cur.execute(sql, (v, fpb))
        db_chem.commit()
        #print(fpb,' - ',v, smi_fl, "inserted")

###################################################################################################################

### Step 4: Insert Fingerprints for New Molecules ###

print()
print('##### Insert Structure/Fingerprints Links #####')
print()

# db_chem = psycopg2.connect(host = "localhost", dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public.fingerprints WHERE fp_type = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(fpdblist)

fpid = {}

for m in fpdblist:
    # print(m)
    i = m[0]
    j = m[3]
    # print(i,j)
    fpid[j]=i
    
# print(fpid)

cur = db_chem.cursor()
sql = 'INSERT INTO public.structure_fingerprint (structure_id,fp_id,fp_count) VALUES (%s,%s,%s);'

# Insert Notification
print()
if len(fp_allmol) == 0:
    print('No new structure/fingerprint inserts')
else:
    print('New structure/fingerprint links')
print()

for m in fp_allmol:
    # j = m[1]
    i = fpid[m[1]]
    cur.execute(sql, (m[0],i,m[2]))
    db_chem.commit()
    print(m,i)
    
db_chem.close()
    
###################################################################################################################

### Step 5: Update Shannon Entropies ###
print()
print('##### Shannon Update #####')
print()

# This is part of netprepper
#    import pandas as pd
#    from scipy import stats as scistat

# import sys
# print(sys.path)
# set PYTHONPATH to proper directory set holds private modules and packages (here pkg_mod)
import netprepper

# SELECT fingerprint length and append to fp_dict

if len(fp_allmol) == 0:
    print('No new structure/fingerprint inserts')
    print()
else:
    #print('New structure/fingerprint links')
    #db_chem = psycopg2.connect(host = "localhost", dbname=db.chem_db, user="postgres", password="postgres")
    db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
    cur = db_chem.cursor() 
    sql = 'SELECT sf.structure_id, sf.fp_id, sf.fp_count, fp.fp_shannon FROM public.structure_fingerprint sf \
    JOIN public.fingerprints fp ON sf.fp_id = fp.fingerprint_id WHERE fp.fp_type = 1'
    cur.execute(sql)
    fp_list = cur.fetchall()
    # print(fp_list)

    fp_dict = collections.defaultdict(list)
    fp_mol = collections.defaultdict(list)

    for m in fp_list:
        # print(m, m[1],m[2])
        fp_dict[m[1]].append(m[2])
        fp_mol[m[0]].append(m[1])

    # n_fp = 0
    print(len(fp_dict),'fingerprints')
    print(len(fp_mol),'structures')
    print()

    n_fp = len(fp_mol)

    #for fpb, i in fp_dict.items():
    #    n_fp = max(n_fp, len(i))
        # print (fpb, i, n)


    # update entropy field in Fingerprints    
    
    sql = 'UPDATE public.fingerprints SET fp_shannon = %s WHERE fingerprint_id = %s'
    cur = db_chem.cursor()
    for fpb, i in fp_dict.items():
        ns = netprepper.shannon(i,n_fp)
        # print(fpb,' - ',netprepper.shannon(i,n))
        # print(fpb,' - ',ns[0],' - ',ns[1])
        print(fpb,' - ',ns[1])
        cur.execute(sql, (ns[1], fpb))

    db_chem.commit()
    db_chem.close()

###################################################################################################################

### Step 6: Update Tanimoto ###
print()
print('##### Tanimoto Update #####')
print()

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor() 

#sql = 'select * FROM public.tanimoto'
#cur.execute(sql)
#q = cur.fetchall()
#print(q)
#print()

sql = 'SELECT max(target) FROM public.tanimoto'
cur.execute(sql)
t = cur.fetchone()
#print(tani_max, type(tani_max))
if t is None:
    tani_max = 0
else:
    tani_max = t[0]
#print(tani_max)

cur = db_chem.cursor() 
sql = 'SELECT structure_id, smiles FROM public.structures ORDER BY structure_id' 
cur.execute(sql) 
mollist = cur.fetchall()
#print(mollist)

cur = db_chem.cursor() 
sql = 'SELECT max(structure_id) FROM public.structures' 
cur.execute(sql) 
m = cur.fetchone()
mol_max = m[0]
print('TaniMax : ',tani_max,' - Mol_max : ',mol_max)

if mol_max > tani_max:
    ml = len(mollist)
    tp = mollist[0][0]
    i = 0
    while tp < tani_max:
        tp = mollist[i][0]
        i = i+1
    print(ml,tp, i)
    tp = i
    cur = db_chem.cursor()
    sql = 'INSERT INTO public.tanimoto (source, target, tanimoto) VALUES (%s,%s,%s);'
    for i in range(tp+1, ml):
        m2 = Chem.MolFromSmiles(mollist[i][1])
        fp2 = AllChem.GetMorganFingerprint(m2,2)
        #fp2 = Chem.RDKFingerprint(m2)
        for j in range (0, i):
            m1 = Chem.MolFromSmiles(mollist[j][1])
            fp1 = AllChem.GetMorganFingerprint(m1,2)
            #fp1 = Chem.RDKFingerprint(m1)
            tan = DataStructs.TanimotoSimilarity(fp1,fp2)
            if tan >= .5:
                #print(i, j, tan, mollist[i][0], mollist[i][1], mollist[j][0], mollist[j][1])
                print(mollist[j][0],mollist[i][0], tan)
                cur.execute(sql, (mollist[j][0],mollist[i][0], tan))

db_chem.commit()
db_chem.close()

##### Tanimoto Backfill #####

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres") 

cur = db_chem.cursor() 
sql = 'SELECT structure_id, smiles FROM public.structures ORDER BY structure_id' 
cur.execute(sql) 
mollist = cur.fetchall()
#print(mollist)

cur = db_chem.cursor() 
sql = 'select s.structure_id \
from public.structures s \
where s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.tanimoto fp WHERE f.structure_id = fp.target)) \
AND s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.tanimoto fp WHERE f.structure_id = fp.source)) \
ORDER BY s.structure_id' 
cur.execute(sql) 
m = cur.fetchall()
#print(m)

for n in m[1:]:    
    tm = n[0]
    #print(n[0])
    cur = db_chem.cursor() 
    sql = 'SELECT structure_id, smiles FROM public.structures WHERE structure_id <= '+ str(tm) +' ORDER BY structure_id' 
    cur.execute(sql) 
    mollist = cur.fetchall()
    #print(mollist)
    maxtan = 0
    ml = len(mollist)
    #print(tm, ml)
    #tp = mollist[0][0]
    cur = db_chem.cursor()
    sql = 'INSERT INTO public.tanimoto (source, target, tanimoto) VALUES (%s,%s,%s);'
    m2 = Chem.MolFromSmiles(mollist[ml-1][1])
    fp2 = AllChem.GetMorganFingerprint(m2,2)
        #fp2 = Chem.RDKFingerprint(m2)
    for j in range (0, ml-1):
        m1 = Chem.MolFromSmiles(mollist[j][1])
        fp1 = AllChem.GetMorganFingerprint(m1,2)
            #fp1 = Chem.RDKFingerprint(m1)
        tan = DataStructs.TanimotoSimilarity(fp1,fp2)
        if tan > maxtan:
            #print(mollist[j][0],mollist[ml-1][0], tan, maxtan)
            maxtan = tan
            ms = mollist[j][0]
            mt = mollist[ml-1][0]
    if maxtan>0:
        print('Max Tani : ',ms, mt, maxtan)
        cur.execute(sql, (ms,mt, maxtan))
    else:
        print('Tani failed : ', tm)

db_chem.commit()
db_chem.close()

###################################################################################################################

### Step 7: INSERT MISSING CHEMBL ID (if exist) ###

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
db_chembl = psycopg2.connect(host = db.db_host, dbname=db.chembl_db, user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_properties fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 4))' 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing ChemBL_ID #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    inchi = Chem.inchi.MolToInchiKey(m, options='')
    #print(i, inchi)
    cur = db_chembl.cursor()
    sql = 'SELECT cs.molregno, md.chembl_id FROM public.compound_structures cs \
    JOIN public.molecule_dictionary md ON cs.molregno = md.molregno \
    WHERE cs.standard_inchi_key = \''+inchi+'\''
    #print(sql)
    cur.execute(sql)
    chmol = cur.fetchall()
    if len(chmol) == 0:
        #print()
        print('No CHEMBL: ', i, inchi)
        #print()
    else:
        print(i, chmol)
        a = (4, i[0], chmol[0][1])
        #print(a)
        cp.StructPropInsert(a)
        
db_chembl.commit()

###################################################################################################################

### Step 8: INSERT MISSING INCHI KEYS ###

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_properties fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 5)) LIMIT '+ limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing InChi Keys #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    inchi = Chem.inchi.MolToInchiKey(m, options='')
    a = (5, i[0], inchi)
    #print(a)
    cp.StructPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 9: INSERT MISSING QED ###

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 1)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing QED #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    qed = Chem.QED.default(m)
    a = (1, i[0], qed)
    #print(a)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 9: INSERT MISSING LOGP ###

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 2)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing LogP #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    #qed = Chem.QED.default(m)
    log = Chem.Crippen.MolLogP(m, True)
    a = (2, i[0], log)
    #print(a)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 10: INSERT MISSING FSP3 ###

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 6)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing Fsp3 #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    fsp = Chem.Lipinski.FractionCSP3(m)
    a = (6, i[0], fsp)
    #print(a, j)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 11: INSERT MISSING H-DONOR ###

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 7)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing H-Donor #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    hdon = Chem.Lipinski.NumHDonors(m)
    a = (7, i[0], hdon)
    #print(a, j)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 12: INSERT MISSING H-ACCEPTOR ###

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 8)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing H-ACCEPTOR #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    hacc = Chem.Lipinski.NumHAcceptors(m)
    a = (8, i[0], hacc)
    print(a, j)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 13: INSERT MISSING MOLECULAR WEIGHT ###

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 9)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing Mol Weights #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    mw = round(Chem.Descriptors.MolWt(m), 3)    
    a = (9, i[0], mw)
    #print(a, j)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 14: INSERT MISSING LIPINSKI ###

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 10)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing Rule of Five Counts #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    lip =0
    m = Chem.MolFromSmiles(j)
    mw = round(Chem.Descriptors.MolWt(m), 3)
    log = Chem.Crippen.MolLogP(m, True)
    hacc = Chem.Lipinski.NumHDonors(m)
    hdon = Chem.Lipinski.NumHDonors(m)
    if mw > 500:
        lip = lip + 1
    if log > 5:
        lip = lip + 1
    if hacc > 10:
        lip = lip + 1
    if hdon > 5:
        lip = lip + 1
    a = (10, i[0], lip)
    #print(mw, log, hacc, hdon)
    #print(a, j)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 15: INSERT MISSING OBJECTS FOR STRUCTURES AND UPDATE STRUCTURE TABLE ###

db_chem = psycopg2.connect(host = db.db_host, dbname=db.chem_db, user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'select s.structure_id \
from public.structures s \
where s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM zafrens.objects fp WHERE f.structure_id = fp.obj_ref \
AND fp.obj_class = \'s\')) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing Structure Objects #####')
#print(mollist)
#print()

sql = "INSERT INTO zafrens.objects(obj_ref, obj_class) VALUES("+j+", 's');"
for i in mollist:
    j= str(i[0])
    #print(j)
    sql = "INSERT INTO zafrens.objects(obj_ref, obj_class) VALUES("+j+", 's');"
    print(sql)
    cur.execute(sql)
        
db_chem.commit()

sql = "select z.obj_id, s.structure_id from public.structures s  join zafrens.objects z \
on s.structure_id = z.obj_ref and z.obj_class = 's' and s.obj_id is NULL"
cur.execute(sql)
objlist = cur.fetchall()
print(objlist)

sql = "update public.structures s set obj_id = z.obj_id from zafrens.objects z \
where structure_id = z.obj_ref and z.obj_class = 's' and s.obj_id is NULL"
cur.execute(sql) 
    
db_chem.commit()


###############################################################################################################
print()
print('#####     #####     #####')
print()
print('Finish Time: ',datetime.now())

PostgreSQL database version: ('PostgreSQL 12.8 (Ubuntu 12.8-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit',)
RDKit version:  2020.03.3
Start Time:  2021-11-07 22:31:14.983538

##### Structures without Fingerprints #####

0 new Structures
[]

All Fingerprints by molecule
[]

{}
0 SMILES

0 fingerprints

##### Insert New Fingerprints #####


No new fingerprint inserts


##### Insert Structure/Fingerprints Links #####


No new structure/fingerprint inserts


##### Shannon Update #####

No new structure/fingerprint inserts


##### Tanimoto Update #####

TaniMax :  16289  - Mol_max :  16289

##### Missing ChemBL_ID #####

No CHEMBL:  (117, 'C1SCCSCCSCCSC1') MULBAFLKPUVJKC-UHFFFAOYSA-N
No CHEMBL:  (88, 'OC1C2NCNC2NC(N)N1') QTRRTHMTUZVPTK-UHFFFAOYSA-N
No CHEMBL:  (71, 'C1C=CC=CS1') QMDFJHAAWUGVKQ-UHFFFAOYSA-N
No CHEMBL:  (68, 'C1=CC=CCO1') MGADZUXDNSDTHW-UHFFFAOYSA-N
No CHEMBL:  (51, 'NCCOCCCC(=O)N[C@@H](CC1=CC=C(OP(O)(=O)NP(O)(=O)OP(O)(=O